# Telegram Parser

А сейчас я расскажу и покажу, как из чата в Телеграме можно достать информацию об его участниках.

Сперва нам необходимо зарегестрировать свое телеграм-приложение. Для этого переходим по ссылке https://my.telegram.org/apps 

Заполняем форму и нажимаем на <b>Create application</b>.

Далее попросят ввести номер телефона для аутефикации по высылаемому паролю. Не паникуем и вводим личные данные. Этот шаг обязателен! 

В появившемся окошке App Configuration находим два поля: App api_id и App api_hash.
Чтобы их не потерять, предлагаю сразу же инициализировать соответствующие переменные своими значениями:

In [7]:
u_id = 123456
u_hash = 'plzEnterApiHash'

А дальше в бой вступает библиотека pyrogram. 

In [29]:
!pip install pyrogram
!pip install tgcrypto

Посмотрим, что тут у нас:

<b>chat</b> - название чата или его ID

<b>string_format</b> - формат строки для вывода

функция <b>to_file</b> для записи результатов в файл

функция <b>to_df</b> для записи результатов в датафрейм

In [14]:
import pandas as pd
import numpy as np
import time
import json
from pyrogram import Client
from pyrogram.errors import FloodWait


app = Client('session', api_id = u_id, api_hash = u_hash) 

chat = 'odsminsk' 
string_format = 'ID: {id} \nИмя: {first_name} \nФамилия:\
{last_name}\nUsername: {username}\nСтатус: {status}\n\n' 


def parser(id):
    members = []
    offset = 0
    limit = 200

    while True:
        try:
            chunk = app.get_chat_members(id, offset)
        except FloodWait as e:
            time.sleep(e.x)
            continue
        if not chunk:
            break
            
        members.extend(chunk)
        offset += len(chunk)
    return members


def text_format(data, template):
    data = json.loads(str(data))
    data['user'].setdefault('first_name', '-') 
    data['user'].setdefault('last_name',  '-')
    data['user'].setdefault('username',  '-')
    
    return template.format(id=data['user']['id'],
                           first_name=data['user']['first_name'],
                           last_name=data['user']['last_name'],
                           username=data['user']['username'],
                           status=data['status'])


def to_file(data, template_format, path):
    with open(path, 'w', encoding='utf8') as file:
        file.writelines('Количество пользователей: {0}\n\n'.format(len(data)))
        file.writelines([text_format(user, template_format) for user in data])
        

def to_df(data):
    uid = [user['user']['id'] for user in data]
    first_name = [user['user']['first_name'] for user in data]
    last_name = [user['user']['last_name'] for user in data]
    username = [user['user']['username'] for user in data]
    
    df = pd.DataFrame();
    df['ID'] = pd.Series(uid)
    df['FIRST_NAME'] = pd.Series(first_name)
    df['lAST_NAME'] = pd.Series(last_name)
    df['USERNAME'] = pd.Series(username)
      
    return df


with app:
    data = parser(chat)
    to_file(data, string_format, 'out_{0}.txt'.format(chat))
    print('Сбор данных окончен')
    
    df = to_df(data)
    print(df)



Сбор данных окончен
            ID FIRST_NAME      lAST_NAME              USERNAME
0    525544142      Darya           None             mylenrise
1    455880722       Варя           None              varya143
2    379565659       Nick         Vlasov            nick_trick
3    115961604     Arseny     Kravchenko            arsenyinfo
4    317026653      Boris     Iosilevsky            mrloregood
..         ...        ...            ...                   ...
512  200751630     Andrei      Klimovich               kwetril
513  427025811      Denis     Kuratovich              krtdenis
514  100933478  Alexander             La               samikb2
515   33796598   Yaroslav       Otnyukov              krasgang
516  346481647      Tanya  Solodovnikova  tatianasolodovnikova

[517 rows x 4 columns]


При первом запуске следующего кода опять попросят ввести номер телефона и ввести пришедший код подтверждения. Делать нечего, придется!

Когда программа отработает, результаты найдем в появившемся файле <b>out_chatname.txt</b>

По итогу, нам удалось вытянуть из чата имена, фамилии, юзернеймы и статусы пользователей. 


### ВСЁ